In [5]:
import os

import qcdb
import openbabel
import pybel

import numpy as np

In [6]:
Z_to_E = {
    1: 'H',
    6: 'C',
    7: 'N',
    8: 'O',
    9: 'F',
    16: 'S',
    17: 'Cl',
}

obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("xyz", "pdb")
dtype = np.float32

In [ ]:
for key in ['HBC6', 'S22', 'S22by7', 'S66', 'UBQ', 'SSI', 'HSG', 'BBI', 'ACHC', 'JSCH',  ]: # ,, 'HBC1', 'NBC1', 
    db = qcdb.Database(key)
    key = db.dbse
    db.write_xyz_files()
    pd_db = db.export_pandas()
    data = {}
    for system_name, size_a, geometry, energy, description, sapt_disp, sapt_es, sapt_ex, sapt_ind in \
        zip(pd_db.QcdbSys, pd_db.MonA, pd_db.Geometry, pd_db.Benchmark, pd_db.System, \
            pd_db['SAPT DISP ENERGY'], pd_db['SAPT ELST ENERGY'], pd_db['SAPT EXCH ENERGY'], pd_db['SAPT IND ENERGY']):
        try:
            with open(key + '_xyzfiles/{}-monoA-unCP.xyz'.format(system_name), 'r') as file:
                line = next(file)
                line = next(file)
                charge_a = int(line.split()[0])            
            with open(key + '_xyzfiles/{}-monoB-unCP.xyz'.format(system_name), 'r') as file:
                line = next(file)
                line = next(file)
                charge_b = int(line.split()[0])
            if charge_a == 0 and charge_b == 0:
                data[system_name] = {}
                data[system_name]['ref_energy'] = dtype(energy * 4.184) # in kJ/mol!
                data[system_name]['description'] = system_name
                data[system_name]['coordinates_dimer'] = geometry.astype(dtype)#[:, 1:]
                data[system_name]['coordinates_monomer_1'] = geometry[:size_a, 1:].astype(dtype)
                data[system_name]['coordinates_monomer_2'] = geometry[size_a:, 1:].astype(dtype)
                Z = np.array(geometry[..., 0], dtype=np.int32)
                eles = np.array([Z_to_E[e] for e in Z], dtype='<U1')    
                #eles = np.array([str(e).decode() for e in eles], dtype=str)
                data[system_name]['elements_monomer_1'] = eles[:size_a]
                data[system_name]['elements_monomer_2'] = eles[size_a:]
                try:
                    data[system_name]['sapt_es'] = dtype(sapt_es * 4.184)
                    data[system_name]['sapt_ex'] = dtype(sapt_ex * 4.184)
                    data[system_name]['sapt_ind'] = dtype(sapt_ind * 4.184)
                    data[system_name]['sapt_disp'] = dtype(sapt_disp * 4.184)
                except:
                    pass
        except:
            continue

        
    np.save('data_' + key, data)
    print(len(data))